In [1]:
import os

In [2]:
%pwd

'/Users/pavithrasenthilkumar/Documents/My_Codes/PROJECTS/Lung_Cancer_Classification_CT_Scan/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/pavithrasenthilkumar/Documents/My_Codes/PROJECTS/Lung_Cancer_Classification_CT_Scan'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/pavi2803/Lung_Cancer_Classification_CT_Scan.mlflow"

In [6]:
import dagshub
import mlflow
dagshub.init(repo_owner='pavi2803', repo_name='Lung_Cancer_Classification_CT_Scan', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as pavi2803

Initialized MLflow to track repo "pavi2803/Lung_Cancer_Classification_CT_Scan"

Repository pavi2803/Lung_Cancer_Classification_CT_Scan initialized!

🏃 View run gifted-sow-822 at: https://dagshub.com/pavi2803/Lung_Cancer_Classification_CT_Scan.mlflow/#/experiments/0/runs/aefad583d74748d0a3676b138d633443
🧪 View experiment at: https://dagshub.com/pavi2803/Lung_Cancer_Classification_CT_Scan.mlflow/#/experiments/0


In [7]:
import tensorflow as tf
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from src.lungClassifier.constants import *
from src.lungClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/pavi2803/Lung_Cancer_Classification_CT_Scan.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    # def log_into_mlflow(self):
    #     mlflow.set_registry_uri(self.config.mlflow_uri)
    #     tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
    #     with mlflow.start_run():
    #         mlflow.log_params(self.config.all_params)
    #         mlflow.log_metrics(
    #             {"loss": self.score[0], "accuracy": self.score[1]}
    #         )
    #         # Model registry does not work with file store
    #         mlflow.keras.log_model(self.model, "model")

    import os

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            # ✅ Save model locally
            local_model_path = "mlruns_model"
            os.makedirs(local_model_path, exist_ok=True)
            full_model_path = os.path.join(local_model_path, "model.h5")
            self.model.save(full_model_path)

            # ✅ Log manually to MLflow
            mlflow.log_artifact(full_model_path, artifact_path="model")


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 102 images belonging to 2 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 1.0000 - loss: 0.0373


🏃 View run clumsy-wasp-43 at: https://dagshub.com/pavi2803/Lung_Cancer_Classification_CT_Scan.mlflow/#/experiments/0/runs/6bcc8bbe91944d8ba7370f7b0a63c981
🧪 View experiment at: https://dagshub.com/pavi2803/Lung_Cancer_Classification_CT_Scan.mlflow/#/experiments/0
